<h1 style="color:#FF5733 ; font-size:50px"> Beneral architecture </h1>
<h3>Gateway configuration</h3>
<h3>Repatriation of the measurements for archiving in a database on the Amazon Web server (EC2)</h3>
<h3>The realization of a Lora WAN gateway based on an uart link between STM32 Raspberry Pi.</h3>


<center><h1 style="color:#FF5733 ; font-size:50px"> Part 1 </h1></center>

<h1 style="color:green  "> 1. Configuration of RASPBERRY Pi 3 </h1>

<h2 style="color:#333399">Tools</h2>

<img src='Picture0.png' style="width:500px;height:300px;"> </img>

<h2 style="color:#333399"> Download RASBIAN </h2>

In [ ]:
We download the image with desktop  based on Debian Stretch

<img src="Picture1.png"></img>

<h2 style="color:#333399"> Download Advanced Ip scanner </h2>

In [ ]:
The Advanced Ip scanner scans all network devices among them the Ip address of Raspberry Pi3.

<img src="Picture2.png"></img>

<h2 style="color:#333399">Find the ip address of Raspberry</h2>

In [ ]:
Ping the ip address of raspberry card.

<img src="Picture3.png"></img>

<h2 style="color:#333399">Putty configuration </h2>

In [ ]:
PuTTY is an SSH client that remote access to another computer.
SSH is an encrypted secure remote login protocol, which uses port 22 by default.

<img src="Picture4.png"></img>

<h2 style="color:#333399"> Remote Desktop Connection </h2>

In [ ]:
Vizialization of the interface of Raspberry pi 3 once we put its IP addess in the computer name field.

<img src="Picture6.png"> </img>

<center><h1 style="color:#FF5733 ; font-size:50px"> Part 2  </h1></center>

<h1 style="color:green  "> 1. Acquisation phase of temperature from GPIO Raspberry Pi </h1>

<h2 style="color:#333399"> Connect the DHT11 sensor to the Raspberry Pi.</h2>

<img src="./pics/GPIO.png" style="width:500px;height:300px;" > 

<h2 style="color:#333399"> What is DHT11 ? </h2>

<img src="./pics/DH11.png" style="width:500px;height:300px;" > 

<h2 style="color:#333399"> Output the temperature readings to an SSH terminal.</h2>

<p> All parameters (temperature, humidity and luminosity, delay, flow rate) are stored
in the same record (table) of a database MySQL. </p>


<h1 style="color:green  "> 2. Python Code for data aquisation </h1>

In [ ]:
import serial
import datetime
import time
import Adafruit_DHT
import mysql.connector

USERNAME ="piste"
PASSWORD = "piste10"
DBNAME = "groupe3"
HOSTNAME = "ec2-54-229-181-249.eu-west-1.compute.amazonaws.com"  
 

con =mysql.connector.connect(host = HOSTNAME ,user= USERNAME,passwd = PASSWORD ,db =  DBNAME, port = 3306)
con.autocommit = True
cursor = con.cursor()
print ("Connected to PISTE10 Database!")
 
while True :
        Humidity, Temperature = Adafruit_DHT.read_retry(11, 27)  # GPIO27 (BCM notation)
        print ("Humidity = {} %; Temperature = {} C".format(Humidity, Temperature))
        cursor.execute("INSERT INTO  agro (Sensor_ID,Humidity,Temperature,Delay,Flow_Rate,time,date) VALUES(%s,%s,%s,%s,%s,%s,%s)" ,
        (1,Humidity,Temperature,0,0,time.strftime(r"%H:%M:%S", time.localtime()),time.strftime(r"%Y-%m-%d", time.localtime())))
        time.sleep(5)
        print ("Done !")
cursor.close()
con.close()

<h2 style="color:#333399"> Explication step by step of the Python code  </h2>

<h3 style="color:#C70039"> Importing Python's needed modules  </h3>

In [ ]:
import serial           #encapsulates the access for the serial port,automatically selects the appropriate backend for python
import datetime         # provides classes to manipulate dates and times
import time             #handle time-related tasks.
import Adafruit_DHT     #library used to communicate with DH11 sensor 
import mysql.connector  #Python needs a MySQL driver to access the MySQL database

<h3 style="color:#C70039"> Installing Adafruit_DHT   </h3>

<p> installing Python library to read the DHT series of humidity and temperature sensors on a Raspberry Pi.

Designed specifically to work with the Adafruit DHT series sensors </p>

In [ ]:
sudo apt-get update                 

sudo apt-get install python-pip

sudo pip install Adafruit_DHT

<h3 style="color:#C70039"> Installing mysql.connector </h3>

<p> MySQL Connector/Python, a self-contained Python driver for communicating with MySQL servers </p>

<h3 style="color:#C70039"> Connection to the Database  </h3>

In [ ]:
USERNAME ="piste"                                                #creation of a user of the database 
PASSWORD = "piste10"                                             #password of the usre to access the database
DBNAME = "groupe3"                                               #table name
HOSTNAME = "ec2-54-229-181-249.eu-west-1.compute.amazonaws.com"  #DNS public (IPv4) of the aws EC2 instance 


<h3 style="color:#C70039"> Setups for EC2 and phpMyAdmin </h3>

In [ ]:
we run the EC2 instance to abtain the DNS public (IPv4) 

<img src="./pics/ec2instance.png" style="width:1000px;height:250px;" > 

<img src="./pics/phpmyadmin.png" style="width:1000px;height:250px;" > 

<h3 style="color:#C70039"> Steps to connect MySQL database in Python using MySQL Connector Python </h3>

<p>1) Install MySQL Connector Python using pip.

2) Use the  mysql.connector.connect()  method of MySQL Connector Python with required parameters to connect MySQL.

3) Use the connection object returned by a  connect()  method to create a cursor object to perform Database Operations.

4) The cursor.execute() to execute SQL queries from Python.

5) Close the Cursor object using a cursor.close() and MySQL database connection using connection.close() after your work    completes.

6) Catch Exception if any that may occur during this process. </p>

<img src="pics/schemaphp.png"> </img>

 <h3 style="color:#C70039"> Understand the Python MySQL Database connection program </h3>

In [ ]:
con =mysql.connector.connect(host = HOSTNAME ,user= USERNAME,passwd = PASSWORD ,db =  DBNAME, port = 3306)

<p> 
<li> mysql.connector.connect() : Using this function we can connect the MySQL Database. </li> 
    <br>
<li>This function accepts four required parameters: Host, Database, User  and Password that we already discussed. </li>

<br>
<li>connect() function established a connection to the  MySQL database from python application and returned a MySQLConnection object.  Then we can use MySQLConnection object to perform various operation on the MySQL Database. </li> </p>

In [ ]:
con.autocommit = True

<p> <li>MySQLdb disables autocommit by default, as required by the DB-API standard (PEP-249).you'll need to do connection.autocommit()  before closing the connection, or else none of your changes will be written to the database. </li>
    <br>
    
<li>Mysql DB supports transaction, "if the database supports an auto-commit feature, this must be initially off" and "closing a connection without committing the changes first will cause an implicit rollback to be performed". </li> </p>

In [ ]:
cursor = con.cursor()
print ("Connected to PISTE10 Database!")

<p> <li> This method returns a cursor object. Using a cursor object, we can execute SQL queries. </li> 
    <br>

<li>The MySQLCursor class instantiates objects that can execute operations such as SQL statements.
Cursor objects interact with the MySQL server using a MySQLConnection object. </li> </p>

In [ ]:
while True :
        Humidity, Temperature = Adafruit_DHT.read_retry(11, 27)  # GPIO27 (BCM notation) and 11 : DH11 sensor 
        print ("Humidity = {} %; Temperature = {} C".format(Humidity, Temperature))

<p> humidity, temperature = Adafruit_DHT.read_retry(sensor, pin)
<p>    </p>  
<li>the variables humidity and tempreture will get the values from the connection to the DH11 sensor  </li>
<br/>

<li>Try to grab a sensor reading.  Use the read_retry method which will retry up
to 15 times to get a sensor reading (waiting 2 seconds between each retry).</li> </p>


In [ ]:
cursor.execute("INSERT INTO  agro (Sensor_ID,Humidity,Temperature,Delay,Flow_Rate,time,date) VALUES(%s,%s,%s,%s,%s,%s,%s)" ,
        (1,Humidity,Temperature,0,0,time.strftime(r"%H:%M:%S", time.localtime()),time.strftime(r"%Y-%m-%d", time.localtime())))

<p> cursor.execute(operation, params=None, multi=False)

This method executes the given database operation (query or command). The parameters found in the tuple or dictionary params are bound to the variables in the operation. Specify variables using %s or %(name)s parameter style (that is, using format or pyformat style). execute() returns an iterator if multi is True. </p>

<p> Using cursor.execute(sql_insert_query, records_to_insert)we are inserting a row (from a List) into the table. </p>

time.sleep(5)
print ("Done !")

<p> wait 5 seconds before every insert in the database </p>

In [ ]:
cursor.close()


<p> Using cursor’s close method we can close the cursor object. Once we close the cursor object, we can not execute any SQL statement.</p>

In [ ]:
con.close()

<p> At last, we are closing the MySQL database connection using a close()  function of MySQLConnection class. </p> 

<h2 style="color:#333399"> Results after executing the python code </h2>

<img src="pics/database.png"> </img>

<h1 style="color:green  "> 2. USART connection between Raspberry Pi and STM32 Lora </h1>

<h2 style="color:#333399"> Raspberry Pi UART GPIO</h2>

<img src="./pics/Pi UART pins.png" > </img>

<h2 style="color:#333399"> STM32 Lora UART GPIO</h2>

<img src="./pics/stm32 uart.jpg" > </img>

<h2 style="color:#333399"> Raspberry Pi Configuration</h2>

<img src="./pics/congig1.PNG" > </img>

<img src="./pics/congig2.PNG" > </img>

<img src="./pics/config3.PNG" > </img>

<img src="./pics/config4.PNG" > </img>

<img src="./pics/config5.PNG" > </img>

<p>After in the raspberry pi configuration, we connect the Rx in Raspberry Pi(GPIO15) with
the Tx of STM32(PA9) and the  Tx in Raspberry Pi(GPIO14) with the Rx of STM32(PA10)</p>

<h2 style="color:#333399"> Pyhton Code of the reception of data and its storage in the database </h2>

In [ ]:
import datetime
import time
import serial
from binascii import hexlify
import mysql.connector

#Connection to AWS database
USERNAME ="piste10"
PASSWORD = "piste10"
DBNAME = "PISTE10"
HOSTNAME = "ec2-108-128-15-84.eu-west-1.compute.amazonaws.com"  
 

con =mysql.connector.connect(host = HOSTNAME ,user= USERNAME,passwd = PASSWORD ,db =  DBNAME, port = 3306)
con.autocommit = True
cursor = con.cursor()
print ("Connected to your Database!")

#getting measurments
ser = serial.Serial(
    port='/dev/ttyUSB0',
    baudrate = 115200,
    parity=serial.PARITY_NONE,
    stopbits=serial.STOPBITS_ONE,
    bytesize=serial.EIGHTBITS,
    timeout=1
    )
print('Connected')
while True :
    data = ser.readline()
    if data:
        print(data)
        x = data.decode("utf-8").split(":")
        print(x)
        ID = x[0]
        temp= x[1]
        hum = x[2]
        lum= x[3]
        cursor.execute("INSERT INTO Agro_Environmental_Parameters(Node_ID,Luminosity,Humidity,Temperature,Time,Date) VALUES(%s,%s,%s,%s,%s,%s)" ,(ID, lum, hum, temp, time.strftime(r"%H:%M:%S", time.localtime()),time.strftime(r"%Y-%m-%d", time.localtime())))
#        cursor.execute("INSERT INTO Agro_Environmental_Parameters(Node_ID,Luminosity,Humidity,Temperature, Time, Date) VALUES(%s,%s, %s, %s,%s,%s)" ,
#        (1,Humidity,Temperature,1,time.strftime(r"%H:%M:%S", time.localtime())))
        time.sleep(2)
        print ("Done !")
cursor.close()
con.close()





<p> This Python code assures the reception of data from the gateway( between the 2 STM32 Lora ) and the insertion of this data in the data base through the Raspberry Pi  </p>

<h2 style="color:#333399"> The format of the received mesurments from the Gateway </h2>

<img src="./pics/reciving.png" >

<p>After receiving the mesurments, and since we are connected to the database we can send the data and insert it in the database</p>

<img src="./pics/finaldatabase.png">

<p>Finally this mesurments will be analyzed by the machine learninig algorithm to determine the decision. </p>